In [12]:
import pandas as pd
data = pd.read_parquet("news_data.parquet")

In [16]:
print(data.head())

      category filename                                            content
0  alt.atheism    49960  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49...
1  alt.atheism    51060  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...
2  alt.atheism    51119  Newsgroups: alt.atheism\nPath: cantaloupe.srv....
3  alt.atheism    51120  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...
4  alt.atheism    51121  Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:51...


In [15]:
print(data.shape)               # Rows and columns
print(data['category'].value_counts())  # How many articles per category
print(data['content'][0])       # View a sample article


(19997, 3)
category
alt.atheism                 1000
comp.graphics               1000
comp.os.ms-windows.misc     1000
comp.sys.ibm.pc.hardware    1000
comp.sys.mac.hardware       1000
comp.windows.x              1000
misc.forsale                1000
rec.autos                   1000
rec.motorcycles             1000
rec.sport.baseball          1000
rec.sport.hockey            1000
sci.crypt                   1000
sci.electronics             1000
sci.med                     1000
sci.space                   1000
talk.politics.guns          1000
talk.politics.misc          1000
talk.politics.mideast       1000
talk.religion.misc          1000
soc.religion.christian       997
Name: count, dtype: int64
Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!p